In [1]:
# Install required packages in Colab
!pip install unsloth transformers datasets accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 4.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.8/299.8 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.6/504.6 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.7/166.7 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8

In [2]:
import torch

print(f"Torch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

Torch version: 2.7.1+cu126
CUDA available: True


In [31]:
import os
import json

# Create directory to save dataset
os.makedirs("data", exist_ok=True)

# Full instruction-based dataset
data = [
    {
        "instruction": "Summarize details about the University of Moratuwa Electrical Engineering department",
        "input": "University: University of Moratuwa.\nDepartment: Electrical Engineering.\nLocation: Sumanadasa Building (Levels 1 & 2), Katubedda, Moratuwa.\nContact: Tel +94 11 2650301 Ext: 3200–3201, Fax +94 11 2650622.",
        "output": "The Electrical Engineering Department is located in the Sumanadasa Building, University of Moratuwa, with contact facilities via phone and fax."
    },
    {
        "instruction": "Summarize the leadership details of the department",
        "input": "Head of Department: Prof. W. D. A. S. Wijayapala (appointed effective 20 Oct 2022). Last HOD was Prof. Udayanga Hemapala (from Jan 2021). Research interests: energy planning, power systems, renewable energy, industrial safety.",
        "output": "Prof. W. D. A. S. Wijayapala is the current HOD, with research in energy and power systems. He succeeded Prof. Udayanga Hemapala."
    },
    {
        "instruction": "List senior professors and quality assurance roles",
        "input": "Senior Professors: Snr. Prof. Sisil Kumarawadu, Snr. Prof. Nalin Wickramarachchi.\nMembers focusing on Quality Assurance: Prof. D. P. Chandima (QA Coordinator). Student Counsellor: Prof. L. N. Widanagama Arachchige.",
        "output": "Senior Professors include Sisil Kumarawadu and Nalin Wickramarachchi. QA is coordinated by Prof. D. P. Chandima, with Prof. L. N. Widanagama Arachchige as Student Counsellor."
    },
    {
        "instruction": "Summarize the professors in the department",
        "input": "Professors: Prof. K.T.M.U. Hemapala; Prof. A.G.B.P. Jayasekara; Prof. D.P. Chandima; Prof. L.N. Widanagama Arachchige; Prof. W.D.A.S. Rodrigo.",
        "output": "The department has five professors: Hemapala, Jayasekara, Chandima, Widanagama Arachchige, and Rodrigo."
    },
    {
        "instruction": "Summarize associate and senior lecturers",
        "input": "Associate Professor: Prof. J.P. Karunadasa. Senior Lecturer Gr. I: Dr. A.M.H.S. Abeykoon; Dr. S.K. Abeygunawardane; Dr. R.M.T. Damayanthi; Dr. W.D. Prasad; Dr. Rasara Samarasinghe.",
        "output": "Prof. J.P. Karunadasa is Associate Professor. Senior Lecturers Gr. I include Abeykoon, Abeygunawardane, Damayanthi, Prasad, and Samarasinghe."
    },
    {
        "instruction": "Summarize Senior Lecturers (Grade II) and lecturers",
        "input": "Senior Lecturer Gr. II: Dr. V. Logeeshan; Dr. R.M. Maheshi Ruwanthika; Dr. S.D.M.S. Gunawardana; Ms. K.A.T. Lasagani; Lecturer (Probationary): Mr. S.R.A. Bolonne.",
        "output": "Senior Lecturers Gr. II are Logeeshan, Ruwanthika, Gunawardana, and Lasagani, with Bolonne as a probationary lecturer."
    },
    {
        "instruction": "List undergraduate academic coordinators",
        "input": "Academic Coordinators (UG BSc): Semester 1 – Dr. W.D. Prasad; Semester 2 – Dr. R. Samarasinghe; Sem 3 – Prof. A.G.B.P. Jayasekara; Sem 4 – Dr. R.M.T. Damayanthi; Sem 5 – Mrs. K.A.T. Lasagani; Sem 6 – Dr. S.D.M.S. Gunawardana; Sem 7 – Dr. R.M.M. Ruwanthika; Sem 8 – Prof. W.D.A.S. Rodrigo.",
        "output": "Each undergraduate semester has a dedicated academic coordinator, including Prasad, Samarasinghe, Jayasekara, Damayanthi, Lasagani, Gunawardana, Ruwanthika, and Rodrigo."
    },
    {
        "instruction": "List postgraduate coordinators",
        "input": "PG Coordinators: MSc/PG Diploma Electrical Engineering – Prof. L. N. Widanagama Arachchige; Electrical Installations – Dr. R.M.T. Damayanthi; Industrial Automation – Prof. A.G.B.P. Jayasekara; PG Certificate Industrial Automation – Dr. L. Velmanickam.",
        "output": "Postgraduate coordinators include Widanagama Arachchige, Damayanthi, Jayasekara, and Velmanickam for various programs."
    },
    {
        "instruction": "Summarize laboratories in the department",
        "input": "Laboratories: Electrical Measurement; Power Systems; Electrical Machines; Controlled Drives; Nuclear Measurements; High Voltage; Computer Systems; Wiring Diagram & Electrical Construction.",
        "output": "The department has labs for measurements, power systems, machines, drives, nuclear measurements, high voltage, computer systems, and electrical construction."
    },
    {
        "instruction": "Summarize student intake and activities",
        "input": "Student Body: approx. 70 undergraduate intake; ~100 postgraduate students annually; department hosts short continuing‑education courses for engineers; strong consultancy links to CEB, SLT and power industry.",
        "output": "The department admits ~70 undergraduates and ~100 postgraduates yearly, offering continuing-education courses and strong industry links."
    },
    {
        "instruction": "Summarize support staff and computing facilities",
        "input": "Support staff: Clerk assisting HOD – Mr. L. Kushan. Computer services: ~35 PCs, NT & UNIX servers, campus LAN and LEARN network in labs and staff rooms.",
        "output": "Support includes a clerk, Mr. Kushan, and about 35 PCs with NT/UNIX servers and campus networking."
    },
    {
        "instruction": "Summarize notable alumni",
        "input": "Notable alumnus: Emeritus Prof. Sam Karunaratne — department’s first professor (1969–2002), founded Computer Engineering dept at UoM; Professor Mahinda Vilathgamuwa — notable power engineering academic trained at UoM and Cambridge.",
        "output": "Notable alumni include Prof. Sam Karunaratne, the department’s first professor and founder of Computer Engineering, and Prof. Mahinda Vilathgamuwa, a leading power engineering academic."
    }
]

# Save as JSONL
with open("data/sample.jsonl", "w") as f:
    for ex in data:
        f.write(json.dumps(ex) + "\n")

print("✅ Full dataset saved at data/sample.jsonl")


✅ Full dataset saved at data/sample.jsonl


In [32]:
from unsloth import FastLanguageModel
from transformers import AutoTokenizer

# Load TinyLlama with 4-bit quantization for efficiency
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    load_in_4bit=True,
)


==((====))==  Unsloth 2025.8.1: Fast Llama patching. Transformers: 4.54.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [34]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "v_proj", "k_proj"],
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    use_gradient_checkpointing=True,
)


Unsloth: Already have LoRA adapters! We shall skip this step.


In [35]:
from datasets import load_dataset

# Load our custom dataset
dataset = load_dataset("json", data_files="data/sample.jsonl", split="train")

# Split dataset into train and test sets (80% train, 20% test)
split_datasets = dataset.train_test_split(test_size=0.2)
train_dataset = split_datasets['train']
test_dataset = split_datasets['test']

# Tokenization function
def tokenize_fn(example):
    prompt = f"### Instruction:\n{example['instruction']}\n\n### Input:\n{example['input']}\n\n### Response:\n{example['output']}"
    tokenized = tokenizer(
        prompt,
        truncation=True,
        padding="max_length",
        max_length=512,
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_fn)


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/12 [00:00<?, ? examples/s]

In [36]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
import torch

training_args = TrainingArguments(
    output_dir="finetuned_model",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    learning_rate=1e-4,
    logging_steps=10,
    num_train_epochs=5,
    bf16=torch.cuda.is_bf16_supported(),
    fp16=not torch.cuda.is_bf16_supported(),
    optim="adamw_torch",
    lr_scheduler_type="cosine",
    report_to="none",
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)


In [37]:
tokenized_train_dataset = train_dataset.map(tokenize_fn)
tokenized_test_dataset = test_dataset.map(tokenize_fn)


Map:   0%|          | 0/9 [00:00<?, ? examples/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [38]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator= data_collator,
)


trainer.train()

print("✅ Training Complete")


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 12 | Num Epochs = 5 | Total steps = 30
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 2 x 1) = 2
 "-____-"     Trainable parameters = 3,063,808 of 1,103,112,192 (0.28% trained)


Step,Training Loss
10,2.219800
20,2.011000
30,2.029700


✅ Training Complete


In [39]:
trainer.save_model("finetuned_model")
tokenizer.save_pretrained("finetuned_model")


('finetuned_model/tokenizer_config.json',
 'finetuned_model/special_tokens_map.json',
 'finetuned_model/chat_template.jinja',
 'finetuned_model/tokenizer.model',
 'finetuned_model/added_tokens.json',
 'finetuned_model/tokenizer.json')

In [40]:
from unsloth import FastLanguageModel
from transformers import AutoTokenizer

model_name_or_path = "finetuned_model"

# Load model (make sure to load with PEFT if needed)
model, tokenizer = FastLanguageModel.from_pretrained(model_name_or_path)


==((====))==  Unsloth 2025.8.1: Fast Llama patching. Transformers: 4.54.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [41]:
example = data[0]

prompt = (
    f"### Instruction:\n{example['instruction']}\n\n"
    f"### Input:\n{example['input']}\n\n"
    "### Response:\n"
)

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

model.eval()
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=128,
        do_sample=True,
        temperature=0.7,
        eos_token_id=tokenizer.eos_token_id,
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


### Instruction:
Summarize details about the University of Moratuwa Electrical Engineering department

### Input:
University: University of Moratuwa.
Department: Electrical Engineering.
Location: Sumanadasa Building (Levels 1 & 2), Katubedda, Moratuwa.
Contact: Tel +94 11 2650301 Ext: 3200–3201, Fax +94 11 2650622.

### Response:
The University of Moratuwa (UM) Electrical Engineering department is located in the Sumanadasa Building (Levels 1 & 2), Katubedda, Moratuwa.
The department has a total of 100 students and 21 faculty members. The electrical engineering curriculum comprises three years of undergraduate study, which includes five semesters of theory and six semesters of laboratory.
The department offers a range of electrical engineering specializations, including power, communication, and control engineering. It also offers courses in digital electronics, analog circuits, and


In [42]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [43]:
results = trainer.evaluate(eval_dataset=tokenized_test_dataset)
print(results)  # shows loss and metrics if any


{'eval_loss': 1.9984277486801147, 'eval_runtime': 0.4668, 'eval_samples_per_second': 6.427, 'eval_steps_per_second': 6.427, 'epoch': 5.0}
